# BEMM459_Week 11 Small Group Teaching
# - py2neo with Python

### <font color="green">Acknowledgement: Tutorial created by refering to sources including py2neo.org and neo4j.com/developer/python/</font>

### <font color="green">Please refer to Neo4J commands from Week 10 cohort teaching - available under <a href=https://github.com/NavonilNM/BEMM459_RDBMS_NoSQL/>BEMM459 GitHub repository - Week 10 </a></font>

# 1. Driver Installation

In [ ]:
# The Neo4j Python driver is officially supported by Neo4j and connects to the database using the binary protocol.
# https://neo4j.com/developer/python/
# HOWEVER,WE ARE NOT USING THIS (we will be using PY2NEO) and so this part of the code is commented (you do not need to use neo4j)

from neo4j import __version__ as neo4j_version
print(neo4j_version)

#from neo4j import __version__ as neo4j_version
print(neo4j_version)

In [ ]:
# Import for the first time
import sys
!{sys.executable} -m pip install py2neo

In [ ]:
# Py2neo is a client library and toolkit for working with Neo4j from within Python applications. 
# The library provides a high level API and an OGM (Object Graph Model), and many other functionalities.

from py2neo import Graph

# Note: You should have a database with the password "BEMM459" - The database should have been started (refer to Week 10 instructions)
graph = Graph("bolt://localhost:7687", auth=("neo4j", "BEMM459"))

# Version and edition
graph.call.dbms.components()

In [ ]:
# OGM is Object-Graph Mapping
from py2neo.ogm import *

class Movie(GraphObject):
    __primarykey__ = "title"

    title = Property()
    tag_line = Property("tagline")
    released = Property()

    actors = RelatedFrom("Person", "ACTED_IN")
    directors = RelatedFrom("Person", "DIRECTED")
    producers = RelatedFrom("Person", "PRODUCED")


class Person(GraphObject):
    __primarykey__ = "name"

    name = Property()
    born = Property()

    acted_in = RelatedTo(Movie)
    directed = RelatedTo(Movie)
    produced = RelatedTo(Movie)
    
alice = Person()
alice.name = "Alice Smith"
print(alice.name)

# 2. Creating Nodes and Relationship

## 2.1 Create node using run()

In [ ]:
#Method 1: Create node and display stats()
#Note: Executing the code block will generate new nodes and relationships

graph.run("CREATE (fr1:Friend) SET fr1.name = 'Hello Friend', fr1.age=25, fr1.address='123 Exeter EX5 6TY'").stats()

## 2.2 Create note and relationship objects using CREATE

In [ ]:
#Method 2: Create node and relationship objects using CREATE
#The two essential building blocks of the property graph model used by Neo4j are the Node and the Relationship. 
#Note: Executing the code block will generate new nodes and relationships

from py2neo import Graph, Node, Relationship

#Transaction begin
tx = graph.begin()

#Creating two nodes, attaching a label 'Person' and adding a property 'name'
p1 = Node("Person", name="Alice")
tx.create(a)
p2 = Node("Person", name="Bob")

#Establishing relationship "KNOWS" to join the two nodes
rel = Relationship(p1, "KNOWS", p2)
tx.create(rel)

#Transaction committed
tx.commit()

#Check to see if graph exists - returns TRUE if graph exists
graph.exists(rel)

rel

In [ ]:
#Method 2: Create node and relationship objects using CREATE

a = Node("Teacher", name="Nav", address="UEBS", phone="not known", email="Nav@UEBS.co.uk")
b = Node("Student", name="Student 1", phone="0745672345", email="S1@Edu.com")
c = Node("Student", name="Student 2", phone="not known", email="S2@Edu.com")

#Creating directional relationships
KNOWS = Relationship.type("_KNOWS_")
ab = KNOWS(a, b)
ba = KNOWS(b, a)
ac = KNOWS(a, c)
ca = KNOWS(c, a)
bc = KNOWS(b, c)
cb = KNOWS(c, b)

#student and teachers all known to each other
teacher_students = ab | ba | ac | ca | bc | cb

#student b knows c and teacher knows only b
#teacher_students = ab | ba | ca | bc

graph.create(teacher_students)

# Print <id>
a.graph, a.identity, b.identity, c.identity

In [ ]:
#Method 2: Adding more data
d = Node("Student", name="Student 3", phone="not known", email="S3@UOE.com", born=1985)
e = Node("Student", name="Student 4", phone="not known", email="S4@UOE.com", born=1990)
f = Node("Student", name="Student 5", phone="deleted", email="S5@UOE.com", born=1983)
g = Node("Student", name="Student 6", phone="hidden", email="S6@Edu.com", born=1992)

KNOWS = Relationship.type("_KNOWS_")
ad = KNOWS(a, d)
da = KNOWS(d, a)
ea = KNOWS(e, a)
fa = KNOWS(f, a)
ga = KNOWS(g, a)


teacher_students = ad | da | ea | fa | ga

graph.create(teacher_students)

## 2.3 Create note and relationship objects using MERGE

In [ ]:
#Method 3: Create node and relationship objects using MERGE
#A Merge operation creates or updates the nodes and relationships of a local subgraph in the remote database
#Note: Executing the code block will NOT generate new nodes and relationships - he MERGE clause ensures that a pattern exists in the graph. Either the pattern already exists, or it needs to be created.

# A simple merge for a new relationship between two new nodes
fr2 = Node("Friend", name="Hello World", age=33)
fr3 = Node("Friend", name="Lion King", age=44)

KNOWS = Relationship.type("KNOWS")
graph.merge(KNOWS(fr2, fr3), "Friend", "name")



In [ ]:
#Method 3: Merging exisitng nodes using MERGE

#We create a third node (of a different type) to which both the original nodes connect.
comp = Node("Company", name="Exeter Technology Solutions")
comp.__primarylabel__ = "Company"
comp.__primarykey__ = "name"
WORKS_FOR = Relationship.type("WORKS_FOR")
graph.merge(WORKS_FOR(fr2, comp) | WORKS_FOR(fr3, comp))

#We merge two nodes created using METHOD 2 using MERGE
graph.merge(WORKS_FOR(a, comp) | WORKS_FOR(b, comp))

# 3. Query
## 3.1 Using run() method

In [ ]:
graph.run("MATCH (a:Student) RETURN a.name, a.phone, a.email LIMIT 5").to_table()

In [ ]:
# Extract the entire result as a list of dictionaries.

graph.run("MATCH (a:Student) RETURN a.name, a.phone, a.email LIMIT 5").data()

In [ ]:
#This method is particularly useful when it is known that a Cypher query returns only a single value.

graph.run("MATCH (a) WHERE a.phone=$x RETURN a.name", x="0745672345").evaluate()

In [ ]:
#Consume and extract the entire result as a pandas.DataFrame.
graph.run("MATCH (a:Student) RETURN a.name, a.phone, a.email LIMIT 5").to_data_frame()

In [ ]:
# Py2neo exposes several logical layers of API on top of the official Python driver. 
# The lowest level Cypher API provides Cypher execution facilities - has Table() object

graph.run("MATCH (a:Person) RETURN a.name, a.born LIMIT 5").to_table()

## 3.2 Node Matching

In [ ]:
# Node Matching - 
# A NodeMatcher can be used to locate nodes that fulfil a specific set of criteria. 
# Typically, a single node can be identified passing a specific label and property key-value pair. 
# However, any number of labels and any condition supported by the Cypher WHERE clause is allowed.
# To refer to the current node within a condition expression, use the underscore character _

from py2neo import Graph, NodeMatcher

matcher = NodeMatcher(graph)
matcher.match("Student", name="Student 1").first()

In [ ]:
# A more comprehensive match using Cypher expressions, the NodeMatch.where()

list(matcher.match("Student").where("_.name =~ 'S.*'"))

In [ ]:
# Orders and limits can also be applied

list(matcher.match("Student").where("_.name =~ 'S.*'").order_by("_.name").limit(2))

In [ ]:
#from py2neo import Graph

from py2neo.matching import *

nodes = NodeMatcher(graph)
nodes.match("Student", email=STARTS_WITH("S1")).all()

In [ ]:
nodes.match("Student", email=ENDS_WITH("UOE.com")).all()

In [ ]:
nodes.match("Student", phone=CONTAINS("not")).all()

In [ ]:
nodes.match("Student", name=LIKE(".*1.*")).all()

In [ ]:
nodes.match("Student", email=IN(["S3@UOE.com", "S4@UOE.com", "S6@UOE.com"])).all()

In [ ]:
nodes.match("Student", born=AND(GE(1980), LE(1990))).all()

In [ ]:
nodes.match("Student", email=OR(STARTS_WITH("S3"), ENDS_WITH("UOE.com"))).all()

# 3.3 Count Functions

In [ ]:
graph.evaluate("MATCH (a:Teacher) RETURN count(a)")

In [ ]:
# Counting the nodes
len(graph.nodes.match("Friend", age=25))

In [ ]:
# Counting the nodes
len(graph.nodes)

# 4. Object-Graph Mapper

## 4.1 Create Objects (OGM)

In [ ]:
class Movie(GraphObject):
    __primarykey__ = "title"

    title = Property()
    tag_line = Property("tagline")
    released = Property()

    actors = RelatedFrom("Person", "ACTED_IN")
    directors = RelatedFrom("Person", "DIRECTED")
    producers = RelatedFrom("Person", "PRODUCED")
    
    #Labels
    comedy=Label()
    action=Label()


class Actor(GraphObject):
    __primarykey__ = "name"

    name = Property()
    born = Property()

    acted_in = RelatedTo(Movie)
    directed = RelatedTo(Movie)
    produced = RelatedTo(Movie)

In [ ]:
peter = Actor()
peter.name = "Peter Pan"

#Injecting node to graph
graph.push(peter)

#Print
peter.__node__

In [ ]:
lionking = Movie()

#Query label
lionking.comedy


In [ ]:
#Setting label to true
lionking.comedy = True

lionking.title = "Lion King"

#Injecting node to graph
graph.push(lionking)

## 4.2 Update Objects (OGM)

In [ ]:
peter = peter.__node__
peter.born= 1960
peter.name = "Not longer Peter Pan"
graph.push(peter)

## 4.3 Deleting Objects (OGM)

In [ ]:
graph.delete(peter)
graph.delete(lionking)